# Max activating examples for 10.7 (by norm projection to logits)

Want to see where head 10.7 is most useful!

In [ ]:
from transformer_lens.cautils.notebook import *

In [ ]:
from transformer_lens.rs.callum.max_activating_exploration import print_best_outputs, find_best_improvements

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)
model.set_use_attn_result(True)

In [ ]:
data = get_webtext(seed=6)

In [ ]:
LAYER_IDX, HEAD_IDX = (10, 7)
W_U = model.W_U.clone()
HEAD_HOOK_NAME = utils.get_act_name("result", LAYER_IDX)

NUM_PROMPTS = 100
BATCH_SIZE = 10

In [ ]:
def hook_to_ablate_head(head_output: Float[Tensor, "batch seq_len head_idx d_head"], hook: HookPoint, head = (LAYER_IDX, HEAD_IDX)):
    assert head[0] == hook.layer()
    assert "result" in hook.name
    head_output[:, :, head[1], :] = 0
    return head_output

## How does 10.7 affect the logits when it's ablated?

We can see cross-entropy loss increases by 0.01 on average when this head is ablated. That might seem like not a lot, but it's actually not far off distribution to other late-stage heads.

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []

for i in tqdm(range(NUM_PROMPTS)):
    # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
    new_str = data[i]
    new_str_tokens = model.to_str_tokens(new_str)
    tokens = model.to_tokens(new_str)
    # tokens = t.stack(tokens).to(device)
    loss = model(tokens, return_type="loss", loss_per_token=True)
    ablated_loss = model.run_with_hooks(tokens, return_type="loss", loss_per_token=True, fwd_hooks=[(HEAD_HOOK_NAME, hook_to_ablate_head)])
    loss_list.append(loss)
    ablated_loss_list.append(ablated_loss)
    str_token_list.append(new_str_tokens)


all_loss = t.cat(loss_list, dim=-1).squeeze()
all_ablated_loss = t.cat(ablated_loss_list, dim=-1).squeeze()

hist(
    all_ablated_loss - all_loss,
    title="Difference in loss after ablating (positive ⇒ loss increases)",
    labels={"x": "Difference in cross-entropy loss"},
    template="simple_white",
    add_mean_line=True,
    width=1000,
    nbins=200
)

In [ ]:
total_num_tokens = sum(len(i) for i in str_token_list)
top_pct = int(total_num_tokens * 0.01)

best_k_indices, best_k_loss_decrease = find_best_improvements(str_token_list, loss_list, ablated_loss_list, k=top_pct)
worst_k_indices, worst_k_loss_decrease = find_best_improvements(str_token_list, loss_list, ablated_loss_list, k=top_pct, worst=True)

In [71]:
caches_and_tokens = print_best_outputs(
    best_k_indices[:3],
    best_k_loss_decrease[:3],
    hook = (HEAD_HOOK_NAME, hook_to_ablate_head),
    model = model,
    data = data,
    n = 3,
    random = False,
    return_caches = False,
)

Guison. "There are now seven stakes and each stake is approximately six to seven congregations-- so that's how 
much it's grown in my lifetime-- just in this area-- so there's been a tremendous amount of growth.""It's been 
exciting to see the growth and people moving to the area," said Nixa Ward Bishop Michael Barker. "But also the 
natural growth from families -- and also those that choose to join the Mormon faith-- those that choose to 
investigate and join themselves to the church-- it's been fun."Before, those of the Mormon faith in Nixa and Ozark 
had to drive to Springfield."Then, as the need grew, we were too crowded in that building," said Bishop Guison. "So
the next choice was what would be the most benefit to members and the community? That's why this location was 
chosen.""It gives a foundation to our faith, a sense of permanence," said Bishop Barker. "There's a lot of 
excitement

                  Correct = ' Barker', Loss decrease from NNMH = 2.433                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Original                                  ┃ Ablated                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Top words:                                │ Top words:                                │
│                                           │                                           │
│ logit = 19.601 | prob = 0.887 | ' Gu'     │ logit = 20.527 | prob = 0.970 | ' Gu'     │
│ logit = 16.911 | prob = 0.060 | ' Barker' │ logit = 15.314 | prob = 0.005 | ' Barker' │
│ logit = 13.880 | prob = 0.003 | ' G'      │ logit = 14.032 | prob = 0.001 | ' G'      │
│ logit = 13.243 | prob = 0.002 | ' N'      │ logit = 13.086 | prob = 0.001 | ' Gh'     │
│ logit = 13.046 | prob = 0.001 | ' Gh'     │ logit = 12.937 | prob = 0.000 | ' Clark'  │
│                                           │                                           │
│ Correct word (predicted at posn 2):       │ Correct word (predicted at posn 2):       │
│                                           │                                           │
│ logit = 16.911 | prob = 0.060 | ' Barker' │ logit = 15.314 | prob = 0.005 | ' Barker' │
└───────────────────────────────────────────┴───────────────────────────────────────────┘

of a rectangular piece with a "half-round dent". The dent is where the opening is located.The mother might eat her
babies, but this mostly happens when she is stressed too much. If you're interested in scorpions in general (or 
other arachnids like tarantulas, vinegaroons, etc), this site has a lot of related photos, nature walks, etc. The 
easiest way to find something of interest is to use google, for example: site:johnbokma.com scorpions.RelatedToday,
after Esme and I had returned from some shopping at Wal-Mart, I discovered that one scorpion was giving birth: a 
Diplocentrus species, probably Diplocentrus melici, which we had captured the 23rd of April, 2006. Later that day 
we captured another female. The latter gave birth some time ago, but shortly after died. My best guess is that it 
somehow got ill, and aborted the scorpions, none of which survived because the

                      Correct = ' scorp', Loss decrease from NNMH = 2.194                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Original                                     ┃ Ablated                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Top words:                                   │ Top words:                                   │
│                                              │                                              │
│ logit = 16.465 | prob = 0.241 | ' baby'      │ logit = 17.162 | prob = 0.290 | ' baby'      │
│ logit = 15.189 | prob = 0.067 | ' pregnancy' │ logit = 16.535 | prob = 0.155 | ' birth'     │
│ logit = 15.044 | prob = 0.058 | ' birth'     │ logit = 15.860 | prob = 0.079 | ' pregnancy' │
│ logit = 14.832 | prob = 0.047 | ' first'     │ logit = 15.167 | prob = 0.040 | ' first'     │
│ logit = 14.586 | prob = 0.037 | ' mother'    │ logit = 14.993 | prob = 0.033 | ' mother'    │
│                                              │                                              │
│ Correct word (predicted at posn 69):         │ Correct word (predicted at posn 344):        │
│                                              │                                              │
│ logit = 11.077 | prob = 0.001 | ' scorp'     │ logit = 9.392 | prob = 0.000 | ' scorp'      │
└──────────────────────────────────────────────┴──────────────────────────────────────────────┘

<|endoftext|>Iran has enlisted tens of thousands of young Shiite men into an armed network that is challenging the 
US across the Middle East. The Trump administration is not prepared.BAGHDAD — Iran has built up a multinational 
network of tens of thousands of young men from across the Middle East, turning them into a well-drilled fighting 
machine that is outgunning the US on the battlefield, as Tehran outsmarts the White House in the corridors of 
power. These men can be found leading the defense of the Syrian president, Bashar al-Assad, recapturing land from 
ISIS in Iraq, and fighting for control of the Yemeni capital of Sanaa. The transnational militia of Shiite men — 
which has no official title — is now the dominant force in the region, enabling Iran to take full advantage in the 
absence of a coherent strategy from the Trump White House. Over six months, BuzzFeed News spoke to researchers, 
officials, and militia fighters who described what they knew about the Iranian program, overseen by the secretive 
Quds

                   Correct = ' Iranian', Loss decrease from NNMH = 0.789                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Original                                   ┃ Ablated                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Top words:                                 │ Top words:                                 │
│                                            │                                            │
│ logit = 16.238 | prob = 0.177 | ' Iranian' │ logit = 15.230 | prob = 0.084 | ' group'   │
│ logit = 15.483 | prob = 0.083 | ' new'     │ logit = 15.198 | prob = 0.082 | ' new'     │
│ logit = 15.312 | prob = 0.070 | ' group'   │ logit = 15.183 | prob = 0.080 | ' Iranian' │
│ logit = 14.742 | prob = 0.040 | ' network' │ logit = 14.868 | prob = 0.059 | ' network' │
│ logit = 14.665 | prob = 0.037 | ' Iran'    │ logit = 14.791 | prob = 0.054 | ' militia' │
│                                            │                                            │
│ Correct word (predicted at posn 1):        │ Correct word (predicted at posn 3):        │
│                                            │                                            │
│ logit = 16.238 | prob = 0.177 | ' Iranian' │ logit = 15.183 | prob = 0.080 | ' Iranian' │
└────────────────────────────────────────────┴────────────────────────────────────────────┘

In [75]:
caches_and_tokens = print_best_outputs(
    best_k_indices[:3],
    best_k_loss_decrease[:3],
    hook = (HEAD_HOOK_NAME, hook_to_ablate_head),
    model = model,
    data = data,
    n = 3,
    random = False,
    return_caches = True,
    names_filter = lambda name: name == utils.get_act_name("pattern", LAYER_IDX),
)
clear_output()

In [79]:
p = Path("/home/ubuntu/Transformerlens/transformer_lens/rs/callum/plots")

window = 100

for i, (cache, tokens) in enumerate(caches_and_tokens):
    
    pattern = cache["pattern", LAYER_IDX][:, HEAD_IDX]
    pattern_sliced = pattern[:, -window:, -window:]
    html = cv.attention.attention_heads(
        attention = pattern_sliced,
        tokens = tokens[-window:],
        attention_head_names = [f"{LAYER_IDX}.{HEAD_IDX}, example {i}"]
    )
    
    with open(str((p / f"temp_file_{i}.html").resolve()), "w") as f:
        f.write(str(html))

    print("".join(tokens[-window:]))
    print("\n" + "=" * 60 + "\n")

 themselves to the church-- it's been fun."

Before, those of the Mormon faith in Nixa and Ozark had to drive to Springfield.

"Then, as the need grew, we were too crowded in that building," said Bishop Guison. "So the next choice was what would be the most benefit to members and the community? That's why this location was chosen."

"It gives a foundation to our faith, a sense of permanence," said Bishop


okma.com scorpions.

Related

Today, after Esme and I had returned from some shopping at Wal-Mart, I discovered that one scorpion was giving birth: a Diplocentrus species, probably Diplocentrus melici, which we had captured the 23rd of April, 2006. Later that day we captured another female. The latter gave birth some time ago, but shortly after died. My best guess is that it somehow got ill, and aborted the


 mascot by the Reveille name in May 2001.

Reveille VII retired in the summer of 2008, and had been living with local residents Paul and Tina Gardner.

Details on services for t

In [ ]:

def clear_plots():
    for i in p.glob("temp_file_*.html"):
        i.unlink()

clear_plots()